In [38]:
%matplotlib inline

import os
import itertools

import numpy as np
import pandas as pd
import xarray as xr

import matplotlib
import matplotlib.pyplot as plt

from IPython import display

In [2]:
display.display(display.Markdown('# B(band, temperature)'))

# B(band, temperature)

In [91]:
def fpath_btable():
    fpath = os.path.join('/chia_cluster/home/jackyu/radiation/crd/LW/src',
                         'lblnew_-_bestfit',
                         'lblnew_-_nref_-_autoabsth_klin_-_gasc_kdesc')
    fpath = os.path.join(fpath, 'btable.dat')
    return fpath


def load_btable(fpath):
    df = pd.read_csv(fpath, sep=r'\s+')
    df = df.set_index(['band', 't'])
    ds = xr.Dataset.from_dataframe(df)
    return ds


def plt_against_t():
    ds = load_btable(fpath_btable())
    
    nbnd = ds.dims['band']
    
    fig, axs = plt.subplots(nrows=1, ncols=2,
                            figsize=(15, 10))
    
    for band in ds.coords['band']:
        ds['b'].sel(band=band).plot(ax=axs[0], 
                                    label='band {}'.format(int(band)))
        ds['slope'].sel(band=band).plot(ax=axs[1], 
                                        label='band {}'.format(int(band)))
        
    [ax.legend() for ax in axs]
    
    axs[0].set_title('b')
    axs[0].set_ylabel('$B(b, t)$')
    axs[0].set_xlabel('temperature (K)')
        
    axs[1].set_title('slope')
    axs[1].set_ylabel('$dB/dt$')
    axs[1].set_xlabel('temperature (K)')    
    return fig
        

    
def into_chunks(l, chunksize):
    return itertools.zip_longest(*(chunksize * [iter(l)]))
    


def vector_to_F77list(array, num_values_per_line=4):
    '''
    '''
    strfmt = '{:15.6e}'
    
    chunks = into_chunks(array, num_values_per_line)
    
    chunks = list(chunks)
    
    lines = []
    for chunk in chunks[:-1]:
        vs = [strfmt.format(v) for v in chunk if v != None]
        line = ','.join(vs)
        line = line + ','
        lines.append(line)
        
    vs = [strfmt.format(v) for v in chunks[-1] if v != None]
    line = ','.join(vs)
    lines.append(line)
    
    return lines
    

    
def btable_to_F77(btable):
    num_values_per_line = 4
    
    last_line = '/)'
    
    varnames = [('b', 'b_planck'), ('slope', 'b_slope')]
    
    
    lines = []
    for dsname, foname in varnames:
        for ib in btable.coords['band']:    
            first_line = '{}(:, {}) = (/'.format(foname, int(ib))
        
            lines_itg = vector_to_F77list(
                btable[dsname].sel(band=ib).values, 
                num_values_per_line=num_values_per_line)
        
            lines_itg = [first_line] + lines_itg + [last_line]
        
            lines_itg_amp = []
            lines_itg_amp.append(6 * ' ' + lines_itg[0])
            for l in lines_itg[1:]:
                lines_itg_amp.append(5 * ' ' + '&' + l)
            
            lines.extend(lines_itg_amp)
    
    fortran = '\n'.join(lines)
    return fortran



def save_s(filename, s):
    clirad_dir = os.path.join('/chia_cluster/home/jackyu/radiation',
                              'clirad-lw/LW/lee_hitran2012_update/')
    
    with open(os.path.join(clirad_dir, filename),
              mode='w', encoding='utf-8') as f:
        f.write(s)
        
    

def script():
    btable = load_btable(fpath_btable())
    s = btable_to_F77(btable)
    save_s('btable.f', s)

In [92]:
script()